In [1]:
import os
import time

%load_ext autoreload

%autoreload 2

os.chdir('/scratch/edk202/hist_w2v')

In [2]:
import rocksdict

from ngram_tools.download_and_ingest_to_rocksdb import download_and_ingest_to_rocksdb
from utils.rocksdb_post_ingest import open_db_read_optimized, run_manual_compaction
from utils.resource_summary import print_resource_summary

E0000 00:00:1755306141.764197 3602869 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755306141.769632 3602869 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755306141.784247 3602869 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755306141.784261 3602869 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755306141.784262 3602869 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755306141.784264 3602869 computation_placer.cc:177] computation placer already registered. Please check linka

In [3]:
!python setup.py clean --all
!python setup.py build_ext --inplace --force

# Reload the optimized modules
from src.reservoir_sampler import reservoir_sampling
from src.count_db_items import count_db_items
from src.ngram_filters import process

Compiling src/ngram_filters.pyx because it changed.
[1/1] Cythonizing src/ngram_filters.pyx
[1/1] Cythonizing src/ngram_filters.pyx
running clean
running clean
removing 'build/temp.linux-x86_64-cpython-311' (and everything under it)
removing 'build/lib.linux-x86_64-cpython-311' (and everything under it)
'build/bdist.linux-x86_64' does not exist -- can't clean it
'build/scripts-3.11' does not exist -- can't clean it
removing 'build'
removing 'build/temp.linux-x86_64-cpython-311' (and everything under it)
removing 'build/lib.linux-x86_64-cpython-311' (and everything under it)
'build/bdist.linux-x86_64' does not exist -- can't clean it
'build/scripts-3.11' does not exist -- can't clean it
removing 'build'
running build_ext
running build_ext
building 'src.reservoir_sampler' extension
creating build/temp.linux-x86_64-cpython-311/src
/ext3/miniforge3/envs/hist_w2v/bin/x86_64-conda-linux-gnu-cc -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunctio

In [4]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
=============================================
Hostname: cm050.hpc.nyu.edu

Job Allocation:
   CPUs: 40
   Memory: 293.0 GB
   Partition: short
   Job ID: 65020982
   Node list: cm050

Physical GPU Hardware:
   No physical GPUs allocated to this job

TensorFlow GPU Recognition:
   TensorFlow can access 0 GPU(s)
   Built with CUDA support: True
=============================================</pre>

# Download

In [ ]:
proj_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/"
db_path = os.path.join(proj_dir, "5grams.db")

download_and_ingest_to_rocksdb(
    ngram_size=5,
    ngram_type='tagged',
    repo_release_id="20200217",
    repo_corpus_id="eng",
    db_path=db_path,
    #file_range=(0, 999),
    workers=16,
    write_batch_size=10000000,
    use_threads=False,
    overwrite=False,
    random_seed=42,
)

In [ ]:
proj_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/"
db_path = os.path.join(proj_dir, "5grams.db")

run_manual_compaction(db_path, optimization="read")

# Inspect Database Contents

In [ ]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db"
db = open_db_read_optimized(db_path)

import struct

# Request (key, value) tuples now that value payload is packed binary
sample = reservoir_sampling(
    db,
    sample_size=10,
    key_type="string",
    progress_interval=100_000_000,
    max_items=2_500_000_000,
    return_keys=True,
)

print("\nSAMPLE")
print("-" * 60)
for i, item in enumerate(sample, 1):
    key, value = item
    freq_tuples = list(struct.iter_unpack('<III', value))
    print(f"{i}. {key}")
    for j, (year, match_count, volume_count) in enumerate(freq_tuples, 1):
        print(f"    {j}: year={year}, match_count={match_count}, volume_count={volume_count}")

In [ ]:
#!rm /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db/LOCK

from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

proj_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/"
db_path = os.path.join(proj_dir, "5grams.db")

db = open_db_read_optimized(db_path)

keys = []
for i, (key, value) in enumerate(db.items()):
    if i >= 10:
        break
    keys.append(key)

for key in keys:
    print(f"Key:         {key}")
    key = tokenize(key)
    print(f"Tokenized:   {key}")
    key = split(key)
    print(f"Split:       {key}")
    key = lower(key)
    print(f"Lowercased:  {key}")
    key = alpha(key)
    print(f"Alpha:       {key}")
    key = shorts(key, 3)
    print(f"Short:       {key}")
    key = stops(key, stopwords)
    print(f"Stopwords:   {key}")
    key = lemmas(key, lemmatizer)
    print(f"Lemmatized:  {key}")
    key = rejoin(key)
    print(f"New Key:     {key}\n")

db.close()

In [ ]:
!rm /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db/LOCK

from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

proj_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/"
db_path = os.path.join(proj_dir, "5grams.db")

db = open_db_read_optimized(db_path)

tag_map = {
    "NOUN": "n",
    "VERB": "v",
    "ADJ": "a",
    "ADV": "r"
}

keys = []
for i, (key, value) in enumerate(db.items()):
    if i >= 10:
        break
    keys.append(key)

for key in keys:
    print(f"Original:   {key}")
    t0 = time.time()
    t1 = time.time(); key = tokenize(key); t2 = time.time()
    t3 = time.time(); key = split(key); t4 = time.time()
    t5 = time.time(); key = lower(key); t6 = time.time()
    t7 = time.time(); key = alpha(key); t8 = time.time()
    t9 = time.time(); key = shorts(key, 3); t10 = time.time()
    t11 = time.time(); key = stops(key, stopwords); t12 = time.time()
    t13 = time.time(); key = lemmas(key, lemmatizer, tag_map); t14 = time.time()
    t15 = time.time(); key = rejoin(key); t16 = time.time()
    print(f"Final:      {key}\n")
    print("Step timings (microseconds):")
    print(f"  Tokenize:    {(t2-t1)*1e6:.1f}")
    print(f"  Split:       {(t4-t3)*1e6:.1f}")
    print(f"  Lowercase:   {(t6-t5)*1e6:.1f}")
    print(f"  Alpha:       {(t8-t7)*1e6:.1f}")
    print(f"  Short:       {(t10-t9)*1e6:.1f}")
    print(f"  Stopwords:   {(t12-t11)*1e6:.1f}")
    print(f"  Lemmatize:   {(t14-t13)*1e6:.1f}")
    print(f"  Rejoin:      {(t16-t15)*1e6:.1f}")
    print(f"  Total:       {(t16-t0)*1e6:.1f}\n")

db.close()


In [ ]:
!rm /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db/LOCK

from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

from nltk.stem import WordNetLemmatizer
import struct
lemmatizer = WordNetLemmatizer()

proj_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/"
db_path = os.path.join(proj_dir, "5grams.db")

db = open_db_read_optimized(db_path)

key, value = next(iter(db.items()))
freq_tuples = list(struct.iter_unpack('<III', value))
freq_tuples = '\t'.join(str(tup) for tup in freq_tuples)
print(f"{key}\t{freq_tuples}")

for key in keys:
    new_key = process(
        key,
        opt_lower=True,
        opt_alpha=True,
        opt_shorts=True,
        opt_stops=True,
        opt_lemmas=True,
        stop_set=stopwords,
        lemma_gen=lemmatizer,
    )
    print(key + " -> " + new_key)

db.close()

"_. A_DET secured_ADJ party_NOUN may_VERB	(1931, 1, 1)	(1960, 3, 3)	(1962, 2, 2)	(1963, 3, 2)	(1964, 1, 1)	(1967, 1, 1)	(1968, 4, 4)	(1970, 1, 1)	(1971, 6, 4)	(1973, 2, 2)	(1975, 1, 1)	(1976, 1, 1)	(1977, 2, 1)	(1981, 7, 3)	(1982, 2, 1)	(1984, 7, 7)	(1985, 7, 7)	(1986, 4, 2)	(1987, 6, 6)	(1988, 1, 1)	(1989, 1, 1)	(1990, 1, 1)	(1991, 1, 1)	(1992, 19, 11)	(1993, 13, 13)	(1994, 14, 10)	(1995, 3, 2)	(1996, 4, 2)	(1997, 3, 3)	(1998, 4, 3)	(1999, 4, 3)	(2000, 6, 3)	(2001, 9, 8)	(2002, 4, 4)	(2003, 1, 1)	(2004, 1, 1)	(2005, 2, 2)	(2006, 1, 1)	(2011, 2, 2)	(2012, 1, 1)	(2013, 4, 3)
"_. A_DET secured_ADJ party_NOUN may_VERB -> <UNK> <UNK> secured party may
"_. A_DET secured_ADJ party_NOUN must_VERB -> <UNK> <UNK> secured party must
"_. A_DET secured_ADJ party_NOUN of_ADP -> <UNK> <UNK> secured party <UNK>
"_. A_DET secured_ADJ party_NOUN who_PRON -> <UNK> <UNK> secured party <UNK>
"_. A_DET securities_NOUN dealer_NOUN occupies_VERB -> <UNK> <UNK> security dealer occupy
"_. A_DET security_NOUN a

In [ ]:
import shutil
import os
import struct
import rocksdict

# Paths
orig_db_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db"
copy_db_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db.processed"

# Copy the DB directory
if os.path.exists(copy_db_dir):
    shutil.rmtree(copy_db_dir)
shutil.copytree(orig_db_dir, copy_db_dir)

In [ ]:
import shutil
import os
import struct
import rocksdict

# Paths
orig_db_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db"
copy_db_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db.processed"

# Copy the DB directory
if os.path.exists(copy_db_dir):
    shutil.rmtree(copy_db_dir)
shutil.copytree(orig_db_dir, copy_db_dir)

# Use the copy for processing
db = rocksdict.Rdict(copy_db_dir, read_only=False)

def freq_merge(existing, new):
    freq_dict = {}
    for tup in struct.iter_unpack('<III', existing + new):
        year, match_count, volume_count = tup
        if year not in freq_dict:
            freq_dict[year] = [match_count, volume_count]
        else:
            freq_dict[year][0] += match_count
            freq_dict[year][1] += volume_count
    agg_tuples = [(year, mc, vc) for year, (mc, vc) in freq_dict.items()]
    return struct.pack(f'<{len(agg_tuples)*3}I', *(x for tup in agg_tuples for x in tup))

db.set_merge_operator(freq_merge)

for key, value in db.items():
    freq_tuples = list(struct.iter_unpack('<III', value))
    new_key = process(
        key,
        opt_lower=True,
        opt_alpha=True,
        opt_shorts=True,
        opt_stops=True,
        opt_lemmas=True,
        stop_set=stopwords,
        lemma_gen=lemmatizer,
    )
    packed = struct.pack(f'<{len(freq_tuples)*3}I', *(x for tup in freq_tuples for x in tup))
    db.merge(new_key, packed)
    del db[key]

db.close()